### Training
---

In [ ]:
from typing import Dict, List, Optional

from nemo.collections import nlp as nemo_nlp
from nemo.utils.exp_manager import exp_manager
from nemo.utils import logging
from nemo.collections.nlp.parts.utils_funcs import tensor2list

import os
import time

import torch
import pytorch_lightning as pl

from omegaconf import OmegaConf

DOCKER = False

if DOCKER:
    HOME_DIR = "/workspace"
else:
    HOME_DIR = "/home/rchen/"

# directory with data converted to nemo format
data_dir = os.path.join(HOME_DIR, "multiatis")

run_name = "test"
config_file = os.path.join("/home/rchen/NeMo/examples/nlp/multi-label-intent-slot-classification/conf/multi-label-intent-slot-classification.yaml")
config = OmegaConf.load(config_file)
config.model.data_dir = data_dir
config.model.validation_ds.prefix = "dev"
config.model.test_ds.prefix = "dev"
config.model.intent_loss_weight = 0.6
config.model.class_balancing = "weighted_loss"
config.model.head.num_output_layers = 2
config.trainer.max_epochs = 5


# checks if we have GPU available and uses it
cuda = 1 if torch.cuda.is_available() else 0
config.trainer.gpus = cuda

# config.trainer.precision = 16 if torch.cuda.is_available() else 32
config.trainer.precision = 32

# for mixed precision training, uncomment the line below (precision should be set to 16 and amp_level to O1):
# config.trainer.amp_level = 'O1'

# remove distributed training flags
config.trainer.accelerator = None

# early_stop_callback = EarlyStopping(monitor='intent_f1', min_delta=1e-1, patience=10, verbose=True, mode='max')

trainer = pl.Trainer(**config.trainer)
config.exp_manager.exp_dir = os.path.join(HOME_DIR, "output/" + run_name)
config.exp_manager.create_checkpoint_callback = True
config.exp_manager.version = time.strftime('%Y-%m-%d_%H-%M-%S')

exp_dir = exp_manager(trainer, config.get("exp_manager", None))
model = nemo_nlp.models.MultiLabelIntentSlotClassificationModel(config.model, trainer=trainer)
trainer.fit(model)

### Evaluation
---

In [ ]:
# extract the path of the best checkpoint from the training, you may update it to any other saved checkpoint file
checkpoint_path = trainer.checkpoint_callback.best_model_path

eval_model= nemo_nlp.models.MultiLabelIntentSlotClassificationModel.restore_from('/home/rchen/output/test/MultiLabelIntentSlot/2022-01-21_08-38-23/checkpoints/MultiLabelIntentSlot.nemo')

# load the model from this checkpoint
#eval_model = nemo_nlp.models.MultiLabelIntentSlotClassificationModel.load_from_checkpoint(checkpoint_path=checkpoint_path)

In [ ]:
eval_model

In [ ]:
queries = [
    'i would like to find a flight from charlotte to las vegas that makes a stop in st. louis',
    'on april first i need a ticket from tacoma to san jose departing before 7 am',
    'how much is the limousine service in boston',
    'How do I fly to China'
]

pred_intents, pred_slots = eval_model.predict_from_examples(queries, config.model.test_ds)
logging.info('The prediction results of some sample queries with the trained model:')

    
for query, intent in zip(queries, pred_intents):
    logging.info(f'Query : {query}')
    logging.info(f'Predicted Intent: {intent}')


In [ ]:
print(config)

In [ ]:
config.keys()

In [ ]:
checkpoint_path

In [ ]:
print(config_file)

In [ ]:
config = OmegaConf.load(config_file)

In [ ]:
checkpoint_path

In [ ]:
!ls /home/rchen/output/test/MultiLabelIntentSlot/

In [ ]:
!ls /home/rchen/output/test/MultiLabelIntentSlot/2022-01-20_11-28-12/checkpoints/MultiLabelIntentSlot.nemo

### Torch Tensor Experiments 
---

In [ ]:
import torch

In [ ]:
lst = [[1,3,1], [4,5,6], [7,8,9]]

In [ ]:
a = torch.tensor(lst)

In [ ]:
a

In [ ]:
a.t()

In [ ]:
a.t()[0]

In [ ]:
for x in a.t()[0]:
    print(x == torch.tensor(1))

In [ ]:
a

In [ ]:
bst = [[1,2,1], [1,5,1], [1,8,1]]

In [ ]:
b = torch.tensor(bst)

In [ ]:
current_label = a[0]
labels_predicted = b[0]

In [ ]:
current_label

In [ ]:
labels_predicted

In [ ]:
(current_label == labels_predicted)[current_label == 1].sum()

In [ ]:
(labels_predicted[current_label] == current_label[labels_predicted]

In [ ]:
current_label

In [ ]:
labels_predicted

In [ ]:
a[0]

In [ ]:
a[0][current_label]

In [ ]:
labels_predicted[current_label]

In [ ]:
current_label[labels_predicted]

In [ ]:
l = [1,2,3]
c = [1,4,3]
d = torch.tensor(l) == 1
e = torch.tensor(c) == 1

In [ ]:
d == 1

In [ ]:
e == 1

In [ ]:
(d == e)[d]

In [ ]:
e

In [ ]:
d